In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
df = pd.read_csv('../input/nba-players-data/all_seasons.csv')
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
conn = sqlite3.connect("../input/nbasql/data.sqlite")

In [ ]:
tables = pd.read_sql("""select *
                        from sqlite_master
                        where type='table';""",conn)
tables

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
nbastats = pd.read_sql("""select * 
                            from all_seasons
                            limit 5;""",conn)
nbastats

## List of Turkish players who have played in the NBA
#### Türk oyuncu listesi

### SQLite

In [ ]:
tr_players = pd.read_sql("""select distinct(player_name) "Player" 
                            from all_seasons
                            where country='Turkey';""",conn)
tr_players

### Python

In [ ]:
df["player_name"][df["country"]=="Turkey"].unique()

## Total number of appearances made by Turkish players
#### Türk oyuncuların toplam maç sayıları

### SQLite

In [ ]:
tr_total_games = pd.read_sql("""select player_name "Player",sum(gp) "Total Games Played"
                            from all_seasons
                            where country="Turkey"
                            group by player_name
                            order by 2 desc;""",conn)
tr_total_games

### Python

In [ ]:
df["gp"][df["country"]=="Turkey"].groupby(df["player_name"]).sum().sort_values(ascending=False).to_frame()

## Career stats of Turkish players
#### Türk oyuncuların kariyer istatistikleri

### SQLite

In [ ]:
tr_avg_stats = pd.read_sql("""select player_name "Player",round(sum(gp*pts)/sum(gp),1)"Avg Points",
                            round(sum(gp*reb)/sum(gp),1)"Avg Rebs",
                            round(sum(gp*ast)/sum(gp),1)"Avg Assists"
                            from all_seasons
                            where country="Turkey"
                            group by player_name
                            order by 2 desc;""",conn)
tr_avg_stats

### Python

In [ ]:
tr_points = round((((df["gp"]*df["pts"])[df["country"]=="Turkey"].groupby(df["player_name"]).sum())/
 (df["gp"][df["country"]=="Turkey"].groupby(df["player_name"]).sum())),1).sort_values(ascending=False)

tr_rebs = round((((df["gp"]*df["reb"])[df["country"]=="Turkey"].groupby(df["player_name"]).sum())/
 (df["gp"][df["country"]=="Turkey"].groupby(df["player_name"]).sum())),1).sort_values(ascending=False)

tr_assists = round((((df["gp"]*df["ast"])[df["country"]=="Turkey"].groupby(df["player_name"]).sum())/
 (df["gp"][df["country"]=="Turkey"].groupby(df["player_name"]).sum())),1).sort_values(ascending=False)

tr_stats = pd.concat([tr_points.rename("Avg Points"),tr_rebs.rename("Avg Rebs"),tr_assists.rename("Avg Assists")],axis=1)
tr_stats

## Points Ranking
#### Sayı Krallığı

## SQLite

In [ ]:
pd.read_sql("""select player_name "Player",round(sum(pts*gp)/sum(gp),1) "Average Points",sum(gp) "Games Played"
                from all_seasons
                group by player_name
                having sum(gp)>=500
                order by 2 desc
                limit 10;""",conn)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622710017946' style='position: relative'><noscript><a href='#'><img alt='Points Ranking ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA1_16227099315990&#47;Sheet1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA1_16227099315990&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA1_16227099315990&#47;Sheet1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622710017946');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Python

In [ ]:
all_points = round((((df["gp"]*df["pts"]).groupby(df["player_name"]).sum())/
 (df["gp"].groupby(df["player_name"]).sum())),1).sort_values(ascending=False)

all_games = (df["gp"]).groupby(df["player_name"]).sum().sort_values(ascending=False)

points_games = pd.concat([all_points.rename("Average Points"),all_games.rename("Games Played")],axis=1)

points_games[points_games["Games Played"]>=500].head(10)

## Top 3 Scorers Each Season
#### Her sezonun sayı sıralamasındaki ilk 3 isim

### SQLite

In [ ]:
pd.read_sql("""with top3scorer as(select player_name "player",season,pts,row_number() 
                over(partition by season order by cast(pts as numeric)desc) "rank",gp "games"
                from all_seasons
                where cast(gp as numeric)>=(
                case when season='1998-99' then 40
                else 70 end))
                select *
                from top3scorer
                where rank<4
                limit 12
            ;""",conn)
# During the 1998-1999 NBA season, each team played only 50 games(due to a lock-out).
# 1998-1999 Sezonunda, lokavt sebebiyle takımlar sadece 50 maç oynamışlardır.

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622711618881' style='position: relative'><noscript><a href='#'><img alt='Top 3 Scorers Each Season ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA2_16227109222830&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA2_16227109222830&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA2_16227109222830&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622711618881');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Python

In [ ]:
df["rank"] = df[(df["season"]=="1998-99") & (df["gp"]>=40) | (df["gp"]>=70)].groupby("season")["pts"].rank(method="first",ascending=False)
df[["player_name","season","pts","rank","gp"]][df["rank"]<=3].sort_values(by=["season","rank"]).head(12).reset_index(drop=True)

## Turkish players who were among top 3 scorers in their respective teams in a season
#### Takımlarında sayı sıralamasında ilk 3'e girmeyi başarmış Türk oyuncular

### SQLite

In [ ]:
pd.read_sql("""with pts_ranking as(select player_name "player",pts,team_abbreviation "team",country,row_number() 
            over(partition by team_abbreviation,season order by cast(pts as numeric)desc)"rank",season
            from all_seasons)
            select *
            from pts_ranking
            where country='Turkey'
            and rank<=3
            order by 5,2 desc
            ;""",conn)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622720523198' style='position: relative'><noscript><a href='#'><img alt='Turkish Players Points Ranking In Their Respective Teams ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA3_16227204301000&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA3_16227204301000&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA3_16227204301000&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622720523198');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Python

In [ ]:
df["rank"] = df.groupby(["team_abbreviation","season"])["pts"].rank(method="first",ascending=False)
df.rename(columns={"player_name":"player","team_abbreviation":"team"})[["player","pts","team","country","rank","season"]][(df["country"]=="Turkey") & (df["rank"]<=3)].sort_values(by=["rank","pts"],ascending=[True,False]).reset_index(drop=True)
# ".rename" done to give each column an alias. inplace=False

## Percentage of team points scored by players
#### Oyuncuların, oynadığı takımların toplam sayısına yüzdesel katkısı

### SQLite

In [ ]:
pd.read_sql("""with ptspercentage as (select player_name as Player,(
                select round(sum(gp*pts))
                from all_seasons as2
                where as1.player_name=as2.player_name) as PlayerPoints,round(sum((
                select sum(gp*pts)
                from all_seasons as3
                where team_abbreviation in (
                select as1.team_abbreviation)
                and season in(
                select as1.season)))) as TeamPoints
                from all_seasons as1
                group by player_name)
select Player,cast(PlayerPoints as int)"PlayerPoints",cast(TeamPoints as int)"TeamPoints",
round((PlayerPoints/TeamPoints)*100,1)|| ' ' || '%' as Percentage
from ptspercentage
where TeamPoints>=30000
order by cast(Percentage as numeric) desc
limit 10
            ;""",conn)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622741575147' style='position: relative'><noscript><a href='#'><img alt='Percentage of team points scored by players ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA4_16227409259460&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA4_16227409259460&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA4_16227409259460&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622741575147');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Python

In [ ]:
player_table = df[["player_name", "team_abbreviation", "season"]].drop_duplicates()
team_points = df.groupby(["team_abbreviation", "season"]).apply(lambda x: np.sum(x.pts * x.gp)).rename("pts").reset_index()
df2 = player_table.merge(team_points, on=["team_abbreviation", "season"])
final = round(df2.groupby("player_name")["pts"].sum()).astype(int).sort_values(ascending=False)
# Note that, unlike SQLite, Pandas does NOT round 0.5 up to 1, hence the different(60841,60840)Team Points values for Malone.

player_points = round(((df["gp"]*df["pts"]).groupby(df["player_name"]).sum())).astype(int)
points_players_teams = pd.concat([player_points.rename("Player Points"),final.rename("Team Points")],axis=1)
points_players_teams["Percentage"] = round((points_players_teams["Player Points"]/points_players_teams["Team Points"])*100,1)
points_players_teams = points_players_teams.sort_values(by="Percentage",ascending=False)
points_players_teams["Percentage"] = points_players_teams["Percentage"].astype(str) + " %"
points_players_teams[points_players_teams["Team Points"]>30000].head(10)

## Percentage of team points scored by Turkish players
#### Türk oyuncuların, oynadığı takımların toplam sayısına yüzdesel katkısı

### SQLite

In [ ]:
pd.read_sql("""with ptspercentage as (select player_name as Player,(
                select round(sum(gp*pts))
                from all_seasons as2
                where as1.player_name=as2.player_name) as PlayerPoints,round(sum((
                select sum(gp*pts)
                from all_seasons as3
                where team_abbreviation in (
                select as1.team_abbreviation)
                and season in(
                select as1.season)))) as TeamPoints,country
                from all_seasons as1
                group by player_name)
select Player,cast(PlayerPoints as int)"PlayerPoints",cast(TeamPoints as int)"TeamPoints",
round((PlayerPoints/TeamPoints)*100,1)|| ' ' || '%' as Percentage
from ptspercentage
where country='Turkey'
order by cast(Percentage as numeric) desc
            ;""",conn)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622741346364' style='position: relative'><noscript><a href='#'><img alt='Percentage of team points scored by players ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA5_16227412065210&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA5_16227412065210&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA5_16227412065210&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622741346364');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Python

In [ ]:
player_table = df[["player_name","country","team_abbreviation", "season"]].drop_duplicates()
team_points = df.groupby(["team_abbreviation", "season"]).apply(lambda x: np.sum(x.pts * x.gp)).rename("pts").reset_index()
df2 = player_table.merge(team_points, on=["team_abbreviation", "season"])
final = round(df2[df2["country"]=="Turkey"].groupby("player_name")["pts"].sum()).astype(int).sort_values(ascending=False)

player_points = round(((df["gp"]*df["pts"])[df["country"]=="Turkey"].groupby(df["player_name"]).sum())).astype(int)
points_players_teams = pd.concat([player_points.rename("Player Points"),final.rename("Team Points")],axis=1)
points_players_teams["Percentage"] = round((points_players_teams["Player Points"]/points_players_teams["Team Points"])*100,1)
points_players_teams = points_players_teams.sort_values(by="Percentage",ascending=False)
points_players_teams["Percentage"] = points_players_teams["Percentage"].astype(str) + " %"
points_players_teams

## Career developments of Kobe and Lebron
#### Kobe ve Lebron'un Kariyer Gelişimleri

### SQLite

In [ ]:
pd.read_sql("""with runningtotals as(select player_name "Player",gp "SeasonGames",pts "SeasonAvg",row_number()
over(partition by player_name order by season)"SeasonNo",gp*pts "SeasonPts",
sum(sum(gp*pts)) over(partition by player_name rows between unbounded preceding and current row)"RunningTotalPts",
sum(sum(gp)) over(partition by player_name rows between unbounded preceding and current row)"RunningTotalGames",season
from all_seasons
where player_name in ('Kobe Bryant','LeBron James')
group by season,player_name)
select Player,cast(round(SeasonPts)as int)"SeasonPts",SeasonGames,SeasonAvg,SeasonNo,cast(round(RunningTotalPts)as int)
"RunningTotalPts",RunningTotalGames,round((RunningTotalPts/RunningTotalGames),1)"RunningAverage",season
from runningtotals
            ;""",conn)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622743596138' style='position: relative'><noscript><a href='#'><img alt='Career developments of Kobe and Lebron ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA6_16227421838990&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA6_16227421838990&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA6_16227421838990&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622743596138');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## Most Improved Players

#### En çok gelişme gösteren oyuncular

### SQLite

In [ ]:
pd.read_sql("""with improved as(select player_name "player",team_abbreviation,gp,pts,
                lag(pts) over(partition by player_name order by season)"prev pts",
                lag(gp) over(partition by player_name order by season)"prev gp",season 
                from all_seasons)
                select player,team_abbreviation "team",pts,"prev pts",(pts-"prev pts")"diff",season
                from improved
                where cast("prev pts" as numeric)>=10
                and cast(gp as numeric)>=60
                and cast("prev gp" as numeric)>=60
                order by diff desc
                limit 15
            ;""",conn)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622878256853' style='position: relative'><noscript><a href='#'><img alt='Most Improved Players ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA7&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA7&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA7&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622878256853');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Python

In [ ]:
df["prev pts"] = df.groupby("player_name")["pts"].shift(1)
df["prev gp"] = df.groupby("player_name")["gp"].shift(1)
df["diff"] = df["pts"] - df["prev pts"]
df[["player_name","team_abbreviation","pts","prev pts","diff","gp","season"]][(df["gp"]>=60) & (df["prev gp"]>=60) & 
(df["prev pts"]>=10)].sort_values(by="diff",ascending=False).reset_index(drop=True).head(15)

## Teams with the highest average net ratings in a season(only top 5 players included)

### SQLite

In [ ]:
pd.read_sql("""with net_rating_order as(select player_name "player",team_abbreviation "team",net_rating,
                row_number() over(partition by season,team_abbreviation order by cast(net_rating as numeric) desc) "rank",
                season
                from all_seasons
                where cast(gp as numeric)>=60
                order by cast(net_rating as numeric) desc),
                top5rating as(select *
                from net_rating_order
                where rank<=5)
                select team,season,round(avg(net_rating),2)"avg net rating"
                from top5rating
                group by season,team
                order by 3 desc
                limit 15
            ;""",conn)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622879920406' style='position: relative'><noscript><a href='#'><img alt='Teams with the highest average net ratings ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA8&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA8&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA8&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622879920406');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Python

In [ ]:
team_rating = df.loc[df["gp"]>=60].copy()
team_rating["rank"] = team_rating.groupby(["season","team_abbreviation"])["net_rating"].rank(method="first",ascending=False)
top5_rating = team_rating[team_rating["rank"]<=5]
top5_rating.groupby(["team_abbreviation","season"])["net_rating"].mean().round(2).sort_values(ascending=False).to_frame().reset_index().head(15)

## Players with the highest average net ratings in a season

### SQLite

In [ ]:
pd.read_sql("""select player_name "player",team_abbreviation "team",net_rating "net rating",season
            from all_seasons
            where cast(gp as numeric)>=60
            order by cast(net_rating as numeric) desc
            limit 15
            ;""",conn)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1622886366327' style='position: relative'><noscript><a href='#'><img alt='Players with the highest average net ratings in a season ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA9&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBA9&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBA9&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1622886366327');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.50)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Python

In [ ]:
df[["player_name","team_abbreviation","net_rating","season"]][df["gp"]>=60].sort_values(by="net_rating",ascending=False).reset_index(drop=True).head(15)

## Total combined points per draft year ranking

### SQLite

In [ ]:
pd.read_sql("""select draft_year "draft year",cast(round(sum(gp*pts))as int)"total pts"
                from all_seasons
                where draft_year!="Undrafted"
                group by draft_year
                order by 2 desc
                limit 10
            ;""",conn)

### Python

In [ ]:
(df["gp"]*df["pts"])[df["draft_year"]!="Undrafted"].groupby(df["draft_year"]).sum().rename("pts").sort_values(ascending=False).round().astype("int").to_frame().head(10)

## Top 3 Scorers of Draft Years starting from 2012

### SQLite

In [ ]:
pd.read_sql("""with rankeddraftyear as(select player_name "player", draft_year "draft year", 
                draft_round"round",draft_number "draft no",
                round(sum(gp*pts)/sum(gp),1)"avg pts",
                row_number()over(partition by draft_year order by sum(gp*pts)/sum(gp)desc )"rank"
                from all_seasons
                where draft_year >= "2012" and draft_year != "Undrafted"
                group by player_name,draft_year)
                select * from
                rankeddraftyear
                where rank<=3
            ;""",conn)

### Python

In [ ]:
draft_info = round((df.groupby(["player_name","draft_year","draft_round","draft_number"]).apply(lambda x: np.sum(x.pts * x.gp)))/(df.groupby("player_name").apply(lambda x: np.sum(x.gp))),1).rename("pts").to_frame()

draft_info = draft_info.reset_index()

draft_info["rank"] = draft_info.groupby("draft_year")["pts"].rank(method="first",ascending=False)

draft_info[(draft_info["draft_year"] != "Undrafted") & (draft_info["draft_year"] >= "2012") & (draft_info["rank"] <= 3)].sort_values(by=["draft_year","pts"],ascending=[True,False]).reset_index(drop=True)

## Most represented countries in the NBA

### SQLite

In [ ]:
pd.read_sql("""select country,count(distinct(player_name))"players"
                from all_seasons
                where country not in ("USA","US Virgin Islands","Bahamas")
                group by country
                order by "players" desc
                limit 15
            ;""",conn)

### Python

In [ ]:
df[~df["country"].isin(["USA","US Virgin Islands","Bahamas"])].groupby("country")["player_name"].nunique().sort_values(ascending=False).head(15).to_frame()
# ~df["country"].isin = not in, nunique = count(distinct)

## Best international scorers

### SQLite

In [ ]:
pd.read_sql("""select player_name "player",country,round(sum(gp*pts)/sum(gp),1)"avg pts",sum(gp)"games"
                from all_seasons
                where country not in ("USA","US Virgin Islands","Bahamas")
                group by player_name,country
                order by "avg pts" desc
                limit 15
            ;""",conn)

### Python

In [ ]:
country_games = df.groupby(["player_name","country"])["gp"].sum()

country_points = round((df.groupby(["player_name","country"]).apply(lambda x: np.sum(x.pts * x.gp)))/(df.groupby(["country","player_name"]).apply(lambda x: np.sum(x.gp))),1)

final = pd.concat([country_points.rename("pts"),country_games],axis=1)

final = final.reset_index()

final[~final["country"].isin(["USA","US Virgin Islands","Bahamas"])].sort_values(by="pts",ascending=False).reset_index(drop=True).head(15)